# Viral load modeling - the predictive version

## Names of group members

put your names here!

## Goal of this assignment

The main goal of this assignment is to model a biological process (namely, the competition between viruses and the human body's immune system) in a mathematical way, and in doing so reproduce the data points that your group was empirically fitting in the last assignment.

## Some background knowledge we need for this model

Viruses multiply in more than one way.  One of the most common is called the [Lytic Cycle](https://en.wikipedia.org/wiki/Lytic_cycle), and the other is the [Lysogenic Cycle](https://en.wikipedia.org/wiki/Lysogenic_cycle).  Both cycles are similar in that the virus takes cells hostage and use the cell's resources to multiply, making many copies of itself.  Once enough new viruses are produced inside the cell it bursts, and the newly-created viruses then are released into the bloodstream to carry on the process and search for new host cells to invade.

[Antiviral drugs](https://en.wikipedia.org/wiki/Antiviral_drug) behave differently than [antibiotics](https://en.wikipedia.org/wiki/Antibiotics) - rather than directly destroying the virus population in a patient, they instead generally inhibit the creation of new viruses by preventing viruses from entering target cells, by preventing the viruses from synthesizing new viruses once they have invaded new cells, or by preventing the release of newly-created viruses from the host cell.

In general, we can think of what happenes to an infected patient that has been administered an antiviral drug using a simple model.  The key points are:

* Viruses multiply rapidly if uninhibited by the body's immune system and infect cells at a rate that is proportional to the number of virions (virus particles), $N_v$, that are in the bloodstream. In other words, $\frac{dN_I}{dt}$, the rate at which the number of cells that are infected ($N_I$) changes, depends on $N_v$ and the time scale for multiplication $t_{mul}$.  $N_v$ in turn depends on the number of infected cells and the number of virions produced per infected cell, $\gamma$.
* As antiviral drugs are administered at a constant rate, it takes some amount of time $T_{crit}$ for the amount of drug in the bloodstream to reach a high enough level that it suppresses the formation of new viruses.  (This time varies from patient to patient, but is typically one day to a few days.)
* After the drug takes effect, we can assume that cell infection immediately stops.  After infection stops, the number of infected cells $N_I$ decreases at a rate $\frac{dN_I}{dt} = -N_{I}/t_{rel}$, where $t_{rel}$ is the time scale on which infected cells release virions into the bloodstream and die.
* Once cells can no longer be infected, virions are released into the bloodstream through the death of previously infected cells.  The rate at which these virions are released  behaves as $\frac{dN_v}{dt} = \gamma N_{I}/t_{rel}$.
* The body clears virions out of the body at a rate that depends on the amount of virions that are in the bloodstream, $\frac{dN_v}{dt} = -N_v/t_{clr}$  ($N_v$ is the number of virions in the bloodstream and $t_{clr}$ is the time scale on which virions are cleared from the body).


## Your mission

You have a mission that will take place in three parts:

1. Using the information above and your whiteboards, create a mathematical model for how the viral load in the bloodstream, $N_v$, varies as a function of time.  Don't use numbers - just symbols!
2. After you are happy with your model (and after you've talked to one of the instructors about it), figure out how to implement it as a computer program.  Do so below! 
3. Compare the shape of the plot created by your model to the data from the HIV viral load project you just completed.  How do they compare?  (Suggestion: assume that all of the time scales in your model are roughly equal, and roughly a day, and vary them from there.)

**INSTRUCTOR NOTES**

It's instructive to look at section 1.2 of Nelson's book.

There is an initial phase where the number of viruses grows exponentially, since $\frac{dN_v}{dt} \propto \frac{dN_I}{dt} \propto \gamma N_v / t_{mul}$.  This leads to $N_v(t) \simeq N_v(0) e^{t/t_{mul}}$, where $N_v(0)$ is the number of virions in the bloodstream at t=0.

Some time $T_{crit}$ after the drug is administered, cell infection stops and the number of infected cells changes as $\frac{dN_I}{dt} = -N_{I}/t_{rel}$.  This leads to the exponential solution $N_I(t) = N_I(0) e^{-t/t_{rel}}$.  (Where $t=0$ is assumed to be at the time that cell infection stops, which is really $T_{crit}$

The total viral load depends on the rate that virions are dumped into the bloodstream by dying cells and cleared from the body by its immune system.  In other words:

$\frac{dN_v}{dt} = -N_v/t_{clr} + \gamma N_{I}/t_{rel}$

Or, taking into account the fact that we know $N_I(t)$ already, with t=0 redefined to be the time where cell infection stops:

$\frac{dN_v}{dt} = -N_v/t_{clr} + \frac{\gamma N_I(0)}{t_{rel}} e^{-t/t_{rel}}$


In [ ]:
# put your computer program here!
%matplotlib inline
import matplotlib.pyplot as plt
import math
import numpy as np

NV0 = 1.0e+4  # number of initial viral cells
NI0 = 1.0e+4  # number of initial infected cells
T_crit = 1.0  # time scale (in days) on which the drug becomes effective
t_clr = 1.0   # time scale (in days) on which 
t_rel = 1.0   # time scale (in days) where inected cells die and release virions
t_mul = 1.0   # growth time of viruses (in days) prior to drug administration
dt = 0.01     # timestep length (in days)
end_time = 10.0 # simulation end time (in days)
gamma = 10.0  # number of virions/infected cell

NV = NV0
NI = NI0

time = []
viral_load = []
infected_cells = []

this_time = 0.0

while(this_time <= end_time):

    if this_time <= T_crit:

        # change in virions w/time 
        dNVdt = gamma * NI0 / t_mul * math.exp(this_time/t_mul) 
    
        NI_inf = NI0 * math.exp(this_time/t_mul)
    
        NV += dNVdt*dt
    
    else:  # this_time > T_crit
    
        dNVdt = gamma*NI_inf/t_rel*math.exp(-(this_time-T_crit)/t_rel) - NV/t_clr
        
        NV += dNVdt*dt
        #NI_inf += 
    
    this_time += dt
    time.append(this_time)
    #infected_cells.append(NI_inf)
    
    viral_load.append(NV)
    
plt.plot(time,viral_load)
plt.yscale('log')

**Computer program notes:**  The students should create either a for loop or a while loop that evolves the system forward in time, and which basically solves $N_v(t)$ with two distinct phases:

1.  Prior to the drug kicking in, where the virus load grows exponentially
2.  After the drug starts working, where the virus load starts falling, first relatively slowly and then more quickly.  (Double exponential model.)

There should be if statements involved, and possibly break/continue statements if they're feeling clever.

## wrapup

Do you have any lingering questions that remain after this project?

put your answers here!

## Turn it in!

Whether you've completed it or not, turn this assignment in to the Day 8 dropbox in the "in-class activities" folder.

Note:  this assignment (as with the previous assignment) was inspired by Nelson's __Physical models of Living Systems__, Chapter 1, and Kinder and Nelson's __A Student's Guide to Python for Physical Modeling__, Chapter 4.